## feature_3によってamountの絶対値あたりの価値は違うという仮説で、特徴をリフトにする

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [22]:
# regist_termをもたせる
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)
base = utils.read_df_pkl('../input/base_term*0*').set_index(key)[['hist_regist_term', 'new_regist_term']]

df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 168.92it/s]


SystemExit: 

In [5]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
df_new = df_new.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

In [12]:
# Save new base
# base = utils.read_df_pkl('../input/base_term*0*').set_index(key)
# base = base.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
# utils.to_df_pkl(df=base.reset_index(), path='../input', fname='base_term')

0it [00:00, ?it/s]

shape: (325540, 17)


3it [00:00,  4.40it/s]


In [25]:
# df_hist = df_hist.join(train_test[['feature_1', 'feature_2']])
# df_new = df_new.join(train_test[['feature_1', 'feature_2']])
df_hist = df_hist.drop(['feature_1', 'feature_2'], axis=1).join(train_test[['feature_1', 'feature_2', 'feature_3']])
df_new = df_new.drop(['feature_1', 'feature_2'], axis=1).join(train_test[['feature_1', 'feature_2', 'feature_3']])

df = df_hist
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth0.shape)

cat1_0 = False
cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]

(2516909, 31)


### NEW 1

In [26]:
prefix = '220_lif'
new_df_list = [new_cat1, new_cat0]
new_fname_list = ['new_cat1', 'new_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'feature_3'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(new_df_list, new_fname_list):
    
    # new
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
    col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
    for col in col_unique:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum']
    aggs['installments'] = ['mean']
    aggs['amount_per_installments'] = ['sum']
    aggs['month_lag'] = ['mean']
    aggs['month_diff'] = ['mean']
    aggs['card_id'] = ['size']
    aggs['date_diff'] = ['mean']
    
    if key not in df.columns:
        df.reset_index(inplace=True)
        df.rename(columns={'index':key}, inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg[lift_level].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
        df_yoko = df_lift[tmp_col].unstack().fillna(0)
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
#             if col.count('feature_'):continue
            if not(col.count('new_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================

100%|██████████| 3/3 [00:00<00:00, 15.44it/s]


Aggregation Start!
(63096, 35)


 67%|██████▋   | 2/3 [00:00<00:00, 15.94it/s]

(45471, 11)
(45471, 2)


100%|██████████| 3/3 [00:00<00:00, 15.56it/s]


...

(325540, 3)
(325540, 3)
(325540, 3)


 67%|██████▋   | 2/3 [00:00<00:00, 14.20it/s]

(284290, 2)


100%|██████████| 3/3 [00:00<00:00, 13.88it/s]


(325540, 3)
(325540, 3)
(325540, 3)


### NEW 2

In [27]:
prefix = '220_lif'
new_df_list = [new_cat1, new_cat0]
new_fname_list = ['new_cat1', 'new_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'feature_3'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(new_df_list, new_fname_list):
    
    # new
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
    col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
    for col in col_unique:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum']
    aggs['installments'] = ['mean']
    aggs['amount_per_installments'] = ['sum']
    aggs['month_lag'] = ['mean']
    aggs['month_diff'] = ['mean']
    aggs['card_id'] = ['size']
    aggs['date_diff'] = ['mean']
    
    if key not in df.columns:
        df.reset_index(inplace=True)
        df.rename(columns={'index':key}, inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg[lift_level].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
        df_yoko = df_lift[tmp_col].unstack().fillna(0)
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
#             if col.count('feature_'):continue
            if not(col.count('new_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================

100%|██████████| 3/3 [00:00<00:00, 15.93it/s]


Aggregation Start!
(63096, 35)


 67%|██████▋   | 2/3 [00:00<00:00, 16.59it/s]

(45471, 11)
(45471, 2)


100%|██████████| 3/3 [00:00<00:00, 16.03it/s]


...

(325540, 3)
(325540, 3)
(325540, 3)


 67%|██████▋   | 2/3 [00:00<00:00, 13.97it/s]

(284290, 2)


100%|██████████| 3/3 [00:00<00:00, 13.36it/s]


(325540, 3)
(325540, 3)
(325540, 3)


### Hist 1

In [28]:
prefix = '220_lif'
auth1_df_list = [auth1_cat1, auth1_cat0]
auth1_fname_list = ['auth1_cat1', 'auth1_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'feature_3'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(auth1_df_list, auth1_fname_list):
    
    # auth1
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
    col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
    for col in col_unique:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum']
    aggs['installments'] = ['mean']
    aggs['amount_per_installments'] = ['sum']
    aggs['month_lag'] = ['mean']
    aggs['month_diff'] = ['mean']
    aggs['card_id'] = ['size']
    aggs['date_diff'] = ['mean']
    
    if key not in df.columns and 'index' not in df.columns:
        df.reset_index(inplace=True)
        df.rename(columns={'index':key}, inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg[lift_level].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
        df_yoko = df_lift[tmp_col].unstack().fillna(0)
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
#             if col.count('feature_'):continue
            if not(col.count('auth1_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================

100%|██████████| 3/3 [00:00<00:00, 16.15it/s]


Aggregation Start!
(1560059, 35)


 67%|██████▋   | 2/3 [00:00<00:00, 16.47it/s]

(121357, 11)
(121357, 2)


100%|██████████| 3/3 [00:00<00:00, 16.39it/s]


...

(325540, 3)
(325540, 3)
(325540, 3)


 67%|██████▋   | 2/3 [00:00<00:00, 13.89it/s]

(320220, 2)


100%|██████████| 3/3 [00:00<00:00, 13.59it/s]


(325540, 3)
(325540, 3)
(325540, 3)


### Hist 2

In [29]:
prefix = '220_lif'
auth1_df_list = [auth1_cat1, auth1_cat0]
auth1_fname_list = ['auth1_cat1', 'auth1_cat0']
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
lift_level = 'feature_3'
ignore_list = list(train_test.columns) + [key, lift_level]

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(auth1_df_list, auth1_fname_list):
    
    # auth1
    df['date_diff'] = (df['hist_purchase_date_max'] - df['purchase_date']).dt.days
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df[f'amount_per_installments'] = df[f'purchase_amount'] / (df[f'installments'] + 1.0)
    
    aggs = {}
    col_unique =['merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    
    for col in col_unique:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum']
    aggs['installments'] = ['mean']
    aggs['amount_per_installments'] = ['sum']
    aggs['month_lag'] = ['mean']
    aggs['month_diff'] = ['mean']
    aggs['card_id'] = ['size']
    aggs['date_diff'] = ['mean']
    
    if key not in df.columns and 'index' not in df.columns:
        df.reset_index(inplace=True)
        df.rename(columns={'index':key}, inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby([key, lift_level]).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(inplace=True)
    
    lift_list = sorted(list(set(df_agg[lift_level].values)))
    feat_cols = [col for col in df_agg.columns if col not in ignore_list]
    
    df_list = []
    for level in lift_list:
        
        tmp_list = []
        for col in feat_cols:
            tmp = df_agg[df_agg[lift_level]==level].reset_index()
            tmp = tmp[[key, col]]
            base_avg = tmp[col].mean()
            tmp.set_index(key, inplace=True)
            tmp[col] /= base_avg
            tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
            tmp_list.append(tmp)
        df_level = pd.concat(tmp_list, axis=1)
        df_level[lift_level] = level
        df_list.append(df_level)
        
    df_lift = pd.concat(df_list, axis=0)
    df_lift = df_lift.reset_index().set_index([key, lift_level])
    print(df_lift.shape)
    for tmp_col in df_lift.columns:
        df_lift[tmp_col] = df_lift[tmp_col].astype('float32')
        df_yoko = df_lift[tmp_col].unstack().fillna(0)
        df_yoko.columns = [f"{lift_level}_{c}_{tmp_col}" for c in df_yoko.columns]
        print(df_yoko.shape)
        
        #========================================================================
        # Save Feature
        base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
        if target in df_yoko.columns:
            df_yoko.drop(target, axis=1, inplace=True)
        df_yoko = base.join(df_yoko)
        for col in df_yoko.columns:
            print(df_yoko.shape)
            if col in ignore_list: continue
#             if col.count('feature_'):continue
            if not(col.count('auth1_cat')):continue
            feature = df_yoko[col].fillna(0).astype('float32').values
            col = col.replace('.', '_')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
            #     utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
            #========================================================================

100%|██████████| 3/3 [00:00<00:00, 10.46it/s]


Aggregation Start!
(1560059, 35)


 67%|██████▋   | 2/3 [00:00<00:00, 17.26it/s]

(121357, 11)
(121357, 2)


100%|██████████| 3/3 [00:00<00:00, 17.01it/s]


...

(320220, 2)


(325540, 3)
(325540, 3)
(325540, 3)


100%|██████████| 3/3 [00:00<00:00, 18.92it/s]

(320220, 2)


(325540, 3)
(325540, 3)
(325540, 3)
